In [ ]:
Model_FileName='XcryptoAi_model.hdf5'
Normalization_File='Normalization_Values.json'
import sys
sys.path.append('a/UltimeTradingBot/Crypto_backtest_tools')
from utilities.get_data import get_historical_from_db
from utilities.backtesting import basic_single_asset_backtest, plot_wallet_vs_asset, get_metrics
import pandas as pd
import ccxt
import matplotlib.pyplot as plt
import ta
import json
import numpy as np
import random
from sklearn.utils import shuffle
import seaborn as sns
from utilities.backtesting import plot_wallet_vs_asset, get_metrics, get_n_columns, basic_multi_asset_backtest, plot_sharpe_evolution, plot_bar_by_month
from utilities.custom_indicators import SuperTrend
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
import os
import gc

pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.filterwarnings('ignore')
PRERR=False
def prerr(err):
    if PRERR:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))

tf.config.list_physical_devices('GPU')

#!nvidia-smi

In [ ]:
#MetaData=get_crypto_metadata(Binance_USDT_HALAL)
MetaData = pd.read_csv("D:\+DATA+\MetaData.csv")
df = pd.read_csv('D:/+DATA+/AICryptoBot/allok_w15_nosell.csv',index_col=0)

In [ ]:
#df=df.drop(columns="sell")
#df=df.drop(columns=["Unnamed: 0"])
#df.to_csv('D:/+DATA+/allok_w15_nosell.csv')

In [ ]:
sys.getsizeof(df)/(1024**2)

## Testing impoted DATA

In [ ]:
df[df['buy']==1]

In [ ]:
df.iloc[1000037]["buy"]

In [ ]:


high_weight=3
df["high"]=(df["open"]+high_weight*df["high"]+df["low"]+df["close"])/(3+high_weight)
df.rename(columns={"high":"price"},inplace = True)
df["BTC_high"]=(df["BTC_open"]+high_weight*df["BTC_high"]+df["BTC_low"]+df["BTC_close"])/(3+high_weight)
df.rename(columns={"BTC_high":"BTC_price"},inplace = True)
df2=df.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])
del(df)
df=df2
#del(df2)
for key in df.keys():
    if key.find("BTC")!=-1 and (key.find("open")!=-1 or
    key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
        df[key]=(df["BTC_price"]-df[key])/df["BTC_price"]
    if key.find("BTC")==-1 and (key.find("open")!=-1 or
    key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
        df[key]=(df["price"]-df[key])/df["price"]

df1=df[df["buy"]==1]
df0=df[df["buy"]==0].iloc[0:len(df1)]
df=pd.concat([df1,df0],axis=0)
del(df1)
del(df0)




In [ ]:
VolRemover=["volume","volume-1","BTC_volume","BTC_volume-1"]
#VolRemover=[]
for key in df.keys():
    if key.find("volume-1_") != -1 :
        VolRemover.append(key)

df=df.drop(columns=VolRemover)

In [ ]:
"volume-1_1day" in df.keys()

In [ ]:
for k in df.keys():print(k, end="  ; ")

In [ ]:
df.keys()

In [ ]:
df.to_csv("cleaned_w15_50percent_novolume.csv")

In [ ]:
# try:
#     df=pd.concat([df1,df0],axis=0).drop(columns=['volume-1', 'volume-1','close','BTC_volume-1', 'BTC_volume-1','BTC_volume-1'])
# except:
#     print("df may be clean")
    
# gc.collect()

In [ ]:
#del(df1)
#del(df0)
gc.collect()

In [ ]:
df=df.reindex(np.random.permutation(df.index))
gc.collect()
df=df.reindex(np.random.permutation(df.index))
gc.collect()
df=df.reindex(np.random.permutation(df.index))
gc.collect()
sys.getsizeof(df)/(1024**2)
df


### Remove volumes

Cheking null values

In [ ]:
gc.collect()
df[df.isnull().any(axis=1)]


checking keys

In [ ]:
for k in df.keys():print(k,end="  ;")
gc.collect()

# starting numpy process

## convert Pandas DataFrame to numpy object

In [ ]:

dt = df.to_numpy()
dt
gc.collect()

In [ ]:
del(df)
gc.collect()

In [ ]:
gc.collect()


In [ ]:
dt.shape

In [ ]:
dt[142110,-1]

### Shuffle the rows Again

In [ ]:
np.random.shuffle(dt)

In [ ]:
sys.getsizeof(dt)/(1024**2)

In [ ]:
#np.savetxt('np_shuffled_cryptodata_w15.csv', dt ,delimiter=',')



In [ ]:
#dt=np.genfromtxt('np_shuffled_cryptodata_w15.csv', delimiter=',')

In [ ]:
index_20percent= int(0.3*len(dt[:,0]))
print(index_20percent)

In [ ]:
XVALIDATION= dt[:index_20percent, :-1]
YVALIDATION= dt[:index_20percent,-1]


In [ ]:
XTRAIN= dt[index_20percent:, 0:-1]
YTRAIN= dt[index_20percent:,-1]

## Normalization

In [ ]:
plt.hist(XTRAIN[:,0])
plt.ylabel("open_1min")
plt.show()

In [ ]:
plt.hist(YTRAIN)
plt.ylabel("Output labels")
plt.show()

plt.hist(YVALIDATION)
plt.ylabel("Output labels")
plt.show()

Tenderalization (mean normalization)

In [ ]:
mean = XTRAIN.mean(axis=0)
XTRAIN -= mean 
std = XTRAIN.std(axis=0)
XTRAIN /= std

XVALIDATION -=mean
XVALIDATION /= std

In [ ]:
Normalization={"mean":mean.tolist(),"std":std.tolist()}
with open(Normalization_File, 'w') as fp:
            json.dump(Normalization, fp,  indent=4)

In [ ]:
plt.hist(XTRAIN[:,0])
plt.ylabel("open column")
plt.show()

In [ ]:
print(XTRAIN.shape)
print(YTRAIN.shape)
print(XVALIDATION.shape)
print(YVALIDATION.shape)

# Neural network model

### Model Generator

In [ ]:
IN_DIM=len(XTRAIN[0,:])

#code genrator
global All_Hist
try:
    with open('Results_history.json') as json_file:
        All_Hist = json.load(json_file)
except:
    All_Hist={0.000001:"code"}

try:
    All_Hist.pop('1e-06')
except:
    print(All_Hist)


The neural network architecture:
- layer 1 : 764 neurons
- layer 2 : 8 neurons
- layer 3 : neurons 

## Model Test

## Compile the model 

In [ ]:
mean.shape

In [ ]:
model = Sequential()
model.add(Dense(int(IN_DIM/2),input_dim=IN_DIM,activation='softplus'))
model.add(Dense(int(IN_DIM/2),activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(int(IN_DIM/5),activation='softplus'))
model.add(Dense(int(IN_DIM/4),activation='softmax'))
model.add(Dense(int(IN_DIM/1),activation='softplus'))
model.add(Dense(1,activation='sigmoid'))
print(model.summary())
#model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
callbacks_a = ModelCheckpoint(filepath = Model_FileName,monitor ='val_accuracy',save_best_only = True, save_weights = True)
callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=20,verbose=1)
history = model.fit(XTRAIN,
                YTRAIN,
                validation_data=(XVALIDATION,YVALIDATION),
                epochs=1200,
                batch_size=5000,
                callbacks=[callbacks_a,callbacks_b])

print('##########################################################################')
print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")


In [ ]:
with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(int(IN_DIM/2),input_dim=IN_DIM,activation='softplus'))
    model.add(Dense(int(IN_DIM/2),activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(int(IN_DIM/5),activation='softplus'))
    model.add(Dense(int(IN_DIM/4),activation='softmax'))
    model.add(Dense(int(IN_DIM/1),activation='softplus'))
    model.add(Dense(1,activation='sigmoid'))
    print(model.summary())
    #model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    callbacks_a = ModelCheckpoint(filepath = Model_FileName,monitor ='val_accuracy',save_best_only = True, save_weights = True)
    callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=20,verbose=1)
    history = model.fit(XTRAIN,
                    YTRAIN,
                    validation_data=(XVALIDATION,YVALIDATION),
                    epochs=1200,
                    batch_size=5000,
                    callbacks=[callbacks_a,callbacks_b])

    print('##########################################################################')
    print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")


In [ ]:
print('##########################################################################')
print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")


In [ ]:
dt[0,-1]

In [ ]:
accuracy = model.evaluate(XVALIDATION, YVALIDATION)


In [ ]:
dt[0:-1]

### Use the model

In [ ]:

# make probability predictions with the model
predictions = model.predict(XVALIDATION)
# round predictions 
rounded = [round(x[0]) for x in predictions]

In [ ]:
rounded[1201]

In [ ]:
YVALIDATION[1201]